In [ ]:
import os
import glob
import keras
import numpy as np
from keras import ops
import seaborn as sns
import tensorflow as tf
from keras import layers
from tensorflow import data as tf_data
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix

keras.utils.set_random_seed(seed=42)

print(keras.__version__)

In [ ]:
def parse_dataset():
    train_points = []
    train_labels = []
#     test_points = []
#     test_labels = []
    class_map = {}
    folders = glob.glob("/kaggle/input/sign-points/dataset/")
    
    letters = "ABCDEFGHIKLMNOPQRSTUVWXY"
    
    for i, letter in enumerate(letters):
        print(f"Processing class: {letter} {i}")
        
        train_directory = f"/kaggle/input/sign-points/dataset/{letter}"
#         test_directory = f"/kaggle/input/sign-points/dataset/{letter}"
        
        class_map[i] = letter
        train_files = glob.glob(train_directory + "/*")
#         test_files = glob.glob(test_directory + "/*")
        
        for train_file in train_files:
            train_points.append(np.load(train_file))
            train_labels.append(i)
            
#         for test_file in test_files:
#             test_points.append(np.load(test_file))
#             test_labels.append(i)
            
        print(len(train_points))
        print(len(train_labels))

    return (
        np.array(train_points),
#         np.array(test_points),
        np.array(train_labels),
#         np.array(test_labels),
        class_map,
    )

In [ ]:
NUM_POINTS = 21
NUM_CLASSES = 24
BATCH_SIZE = 64

train_points, train_labels, CLASS_MAP = parse_dataset()

In [ ]:
# test_dataset = tf_data.Dataset.from_tensor_slices((test_points, test_labels))
# test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)

dataset = tf_data.Dataset.from_tensor_slices((train_points, train_labels))

train_size = int(len(dataset) * 0.75)
validation_size = int(len(dataset) * 0.20)
test_size = int(len(dataset) * 0.05)

dataset = dataset.shuffle(len(train_points))

train_dataset = dataset.take(train_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_size).take(validation_size).batch(BATCH_SIZE)
test_dataset = dataset.skip(train_size+validation_size).take(test_size).batch(BATCH_SIZE)

In [ ]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)


def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

In [ ]:
class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = ops.eye(num_features)

    def __call__(self, x):
        x = ops.reshape(x, (-1, self.num_features, self.num_features))
        xxt = ops.tensordot(x, x, axes=(2, 2))
        xxt = ops.reshape(xxt, (-1, self.num_features, self.num_features))
        return ops.sum(self.l2reg * ops.square(xxt - self.eye))

In [ ]:
def tnet(inputs, num_features):
    # Initialise bias as the identity matrix
    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    # Apply affine transformation to input features
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [ ]:
inputs = keras.Input(shape=(NUM_POINTS, 3))

x = tnet(inputs, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)

outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="pointnet")
model.summary()

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    metrics=["sparse_categorical_accuracy"],
)

history = model.fit(train_dataset, epochs=100, validation_data=validation_dataset)

In [ ]:
model.save("model.keras")
model.save("model2.h5")

In [ ]:
import matplotlib.pyplot as plt

# Extracting the history of training and validation accuracy and loss
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Creating plots
plt.style.use('seaborn-whitegrid') # Use a modern style
colors = ['#377eb8', '#ff7f00']  # A set of colors: blue and orange
linestyles = ['-', '--']  # Solid and dashed lines

fig = plt.figure(figsize=(12, 6))
fig.patch.set_facecolor('#f7f7f7')

# Plotting training and validation accuracy
plt.subplot(1, 2, 1)
plt.title('Training and Validation Accuracy', fontsize=20)
plt.plot(epochs, acc, 'r', label='Training Accuracy', linestyle=linestyles[0], color=colors[0], linewidth=2)
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy", linestyle=linestyles[1], color=colors[1], linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.grid(True, linestyle='--', linewidth=0.2)

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.title('Training and Validation Loss', fontsize=20)
plt.plot(epochs, loss, 'r', label='Training Loss', linestyle=linestyles[0], color=colors[0], linewidth=2)
plt.plot(epochs, val_loss, 'b', label="Validation Loss", linestyle=linestyles[1], color=colors[1], linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.grid(True, linestyle='--', linewidth=0.2)

plt.savefig("training-and-validation-accuracy-and-loss.png")
plt.show()

In [ ]:
data = test_dataset.take(1)

points, labels = list(data)[0]
points = points[:8, ...]
labels = labels[:8, ...]

# run test data through model
preds = model.predict(points)
preds = ops.argmax(preds, -1)

points = points.numpy()

# plot points with predicted class and label
fig = plt.figure(figsize=(15, 10))
for i in range(8):
    ax = fig.add_subplot(2, 4, i + 1, projection="3d")
    ax.scatter(points[i, :, 0], points[i, :, 1], points[i, :, 2])
    ax.set_title(
        "pred: {:}, label: {:}".format(
            CLASS_MAP[preds[i].numpy()], CLASS_MAP[labels.numpy()[i]]
        )
    )
    ax.set_axis_off()
plt.show()

In [ ]:
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import confusion_matrix

y_pred = []  # store predicted labels
y_true = []  # store true labels

for image_batch, label_batch in test_dataset:
   # append true labels
   y_true.append(label_batch)
   # compute predictions
   preds = model.predict(image_batch, verbose=0)
   # append predicted labels
   y_pred.append(np.argmax(preds, axis = - 1))

# convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_true], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

plt.figure(figsize=(30, 20))
fig.patch.set_facecolor('#f7f7f7')

ax = sns.heatmap(metrics.confusion_matrix(correct_labels,predicted_labels), cmap="rocket_r")

ax.set_title('Confusion Matrix', fontsize=20, pad=20)
ax.set_xlabel('Predicted Values', fontsize=14, labelpad=15)
ax.set_ylabel('Actual Values ', fontsize=14, labelpad=15)

letters = "ABCDEFGHIKLMNOPQRSTUVWXY"
## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(list(letters))
ax.yaxis.set_ticklabels(list(letters))

# Improving tick labels for clarity
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', rotation=90, labelsize=14)

plt.savefig("confusion-matrix-1.png")
plt.show()